# Paper Toss

## Importing files and creating the funtion for the arduino 

In [ ]:
import numpy as np
import copy
import ode

import serial, time

In [ ]:
#this function writes a character to the Arduino and receives LR and UD  voltages from the Arduino
def getData():
    ser.write(str.encode(chr(1)))
    line = ser.readline()
    #print(line)
    data = line.split()
    button = int(data[0])
    LR1 = int(data[1])
    UD1 = int(data[2])
    LR2 = int(data[3])
    UD2 = int(data[4])
    #print(str(LR)+"\t"+str(UD))
    return [button, LR1, UD1, LR2, UD2]

#serial port for the Arduino; get the name for the port from the Arduino software
port = 'COM4'

#start the serial port
ser = serial.Serial(port, 9600, timeout=2)

# The following line is necessary to give the arduino time to start
# accepting stuff.
time.sleep(2)

## The Actual Game

In [ ]:
from vpython import *
scene1 = canvas(title='PaperToss',
     width=500, height=600,
     center=vector(0,0,0),userzoom = True,userspin = True)
scene.autoscale=False
r = 10

ball = sphere(pos=vec(0,-100,0),size=vec(r,r,r),color=color.cyan)
arrow = arrow(pos=vec(0,-100,0), axis= 2*vec(0,10,0), color = color.green)
goal = cylinder(pos=vec(0,50, -150),  axis=vec(0,50,10), radius=30, color=color.white, opacity=0.8)
scene1.pause()
dt = 0.0001
t = 0
tmax = 1000
v = vec(0, 5, 5)
scale = 2
xvel = 10
Data = [] 
collision = False
thrown = False
ball.m = 0.75
zvel = -9.81*ball.m
while(not thrown):
    rate(100)
    Data = getData()
    #xaxis movement
    if(Data[0] == 1):
        thrown = True
    yvel = Data[1]
    #print(yvel)
    
    ball.pos.x = ball.pos.x + scale*((yvel-506)/1000)
    arrow.pos = ball.pos
    #conditions on how far the ball can move
    if(ball.pos.x > 100):
        ball.pos.x = 100
        print("You Can't Go That Far!")
    elif(ball.pos.x < -100):
        ball.pos.x = -100
        print("You Can't Go That Far!")
    
    #print(ball.pos.x)
    #velocity of ball (constant)
    if(Data[2] - 516 > 1):
        yvel = scale*((Data[2] - 516)/100+5)
        arrow.axis = scale*vec(0,yvel,0) 
    elif(Data[2] - 516 < -1):
        yvel = scale*((Data[2] - 516)/100+3)
        arrow.axis = scale*vec(0,yvel,0) 

if(thrown):
    #y and z axis movement 
    while(not collision and t < tmax):
        rate(1000)
        ball.pos.y = ball.pos.y + yvel/100
        ball.pos.z = ball.pos.z + zvel/100
        t = t+ dt
        if(goal.pos.y+100 < ball.pos.y or goal.pos.z-100 > ball.pos.z):
            print("You Missed SMH")
            t = tmax
            
        elif(goal.pos.y  < ball.pos.y and goal.pos.y+50  >  ball.pos.y):
            if(goal.pos.x-30 < ball.pos.x and goal.pos.x+30 >  ball.pos.x):
                if(goal.pos.z+20 > ball.pos.z and goal.pos.z-20 <  ball.pos.z):
                    print("You got it!!!")
                    collision = True
        elif(ball.pos.y < -150):
            print("Wrong Way Silly")
            t = tmax
        
            

## Physics for the fan

In [1]:
#force of the fan
p=1.2 #desnity of air 
A = 0.12 #area of the fan blades
v = 15 #velocity of the fan blades
f = (A*p*v**3)/4 #https://www.wired.com/2012/09/modeling-the-force-from-a-fan/ source
#that would be the force on the air from the fans and that should effect the ball but it should be weaker the farther away it is.  thus I can to the conclution of
#error cathing i would need to account for  negative x postion for the ball with an if statement. but that is fine. This would also only be calculated once as the ball is thrown 
#fob = f - (fan.pos.x - ball.pos.x)/100 
#that is commented because fan doesn not exist at this moment but it would just create an error. also the fan balde are technically moving extremly fast in this senario because it has such a small area of effect that it needed to be larger in oder for it to do anythin at all to the ball.
#also since no other forces would be acting on the ball while it is in the x direction it will continue with the same velocity once it leave and it will then be constant. 
#just a little thing for u to know :D thanks for a great semester
#ps will deleted the last comment if i actually make this fully public and plan to keep it up. 